# Machine Learning-Based Prediction of 30-Day Hospital Readmission in Diabetic Patients

## Feature Engineering

In [156]:
import pandas as pd
import numpy as np

from scipy.stats import chi2_contingency

In [157]:
df = pd.read_csv("./data/raw/day-26.csv")

In [158]:
df.shape

(101763, 46)

In [159]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101763 entries, 0 to 101762
Data columns (total 46 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   race                      101763 non-null  object
 1   gender                    101763 non-null  object
 2   age                       101763 non-null  object
 3   admission_type            101763 non-null  object
 4   discharge_disposition     101763 non-null  object
 5   admission_source          101763 non-null  object
 6   time_in_hospital          101763 non-null  int64 
 7   payer_code                101763 non-null  object
 8   medical_specialty         101763 non-null  object
 9   num_lab_procedures        101763 non-null  int64 
 10  num_procedures            101763 non-null  int64 
 11  num_medications           101763 non-null  int64 
 12  number_outpatient         101763 non-null  int64 
 13  number_emergency          101763 non-null  int64 
 14  numb

### Convert Target Column

In [160]:
# Check unique values

print(df['readmitted'].value_counts())

readmitted
NO     54861
>30    35545
<30    11357
Name: count, dtype: int64


In [161]:
# Convert to binary
df['readmitted_binary'] = df['readmitted'].apply(
    lambda x: 1 if x == '<30' else 0
)

In [162]:
# Verify conversion
df['readmitted_binary'].value_counts()

readmitted_binary
0    90406
1    11357
Name: count, dtype: int64

In [163]:
df['readmitted_binary'].value_counts(normalize=True)

readmitted_binary
0    0.888398
1    0.111602
Name: proportion, dtype: float64

### Convert Drug Columns to Numeric

In [164]:
# No      → 0
# Steady  → 1
# Up      → 1
# Down    → 1

# Because the important question is was this drug given or not?

In [165]:
drug_cols = df.columns[20:43]

In [166]:
df[drug_cols] = df[drug_cols].replace({
    'No': 0,
    'Steady': 1,
    'Up': 1,
    'Down': 1
})

C:\Users\dharm\AppData\Local\Temp\ipykernel_37448\3731152250.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[drug_cols] = df[drug_cols].replace({


### Create a Powerful Feature

#### Feature 1: Total Diabetes Drugs Given

In [167]:
df['total_diabetes_drugs'] = df[drug_cols].sum(axis=1)

# How many different diabetes drugs patient is taking.

#### Feature 2: Insulin Flag

In [168]:
df['insulin_flag'] = df['insulin']

# Insulin is very important indicator of severity.

#### Feature 3: Oral Medication Flag

In [169]:
oral_drugs = drug_cols.drop('insulin')

df['oral_med_flag'] = df[oral_drugs].sum(axis=1)
df['oral_med_flag'] = df['oral_med_flag'].apply(lambda x: 1 if x > 0 else 0)

### Medication Change Behavior

In [170]:
# Converting change column from categorical to numeric

df['change'].value_counts()

change
No    54754
Ch    47009
Name: count, dtype: int64

In [171]:
df['change'] = df['change'].map({'No':0,'Ch':1})

In [172]:
df['change'].value_counts()

change
0    54754
1    47009
Name: count, dtype: int64

In [173]:
# Converting diabetesMed from categorical to numeric column

df['diabetesMed'].value_counts()

diabetesMed
Yes    78361
No     23402
Name: count, dtype: int64

In [174]:
df['diabetesMed'] = df['diabetesMed'].map({"Yes":1 , "No":0})

In [175]:
df['diabetesMed'].value_counts()

diabetesMed
1    78361
0    23402
Name: count, dtype: int64

### Create new column as is_polypharmacy

In [176]:
df['is_polypharmacy'] = df['num_medications'].apply(lambda x: 1 if x >= 10 else 0)

### Drop Original 25 Columns

In [177]:
df.drop(columns=drug_cols, inplace=True)

In [178]:
df.drop(columns=['readmitted'], inplace=True)

In [179]:
df.shape

(101763, 27)

### Handling age column

In [180]:
df['age'].value_counts()

age
[70-80)     26066
[60-70)     22482
[50-60)     17256
[80-90)     17197
[40-50)      9685
[30-40)      3775
[90-100)     2793
[20-30)      1657
[10-20)       691
[0-10)        161
Name: count, dtype: int64

In [181]:
df['age'] = df['age'].str.extract('(\d+)').astype(int) + 5

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\dharm\AppData\Local\Temp\ipykernel_37448\2740532608.py:1: SyntaxWarning: invalid escape sequence '\d'
  df['age'] = df['age'].str.extract('(\d+)').astype(int) + 5


### Checking payer_code column

In [182]:
# Applying 'chi2' test and finding 'Cramér's V' on payer_code to find Strength of the column with target column 

In [183]:
df['payer_code'].nunique()

18

In [184]:
# Create a cross-tabulation table
contingency_table = pd.crosstab(df['payer_code'], df['readmitted_binary'])

# Run the test
chi2, p, dof, ex = chi2_contingency(contingency_table)
print(f"P-value: {p}")

P-value: 2.388095995508881e-13


In [185]:
# 'chi2' is the value you got from your test
n = df.shape[0]  # total observations
min_dim = min(contingency_table.shape) - 1
cramers_v = np.sqrt(chi2 / (n * min_dim))

print(f"Cramér's V (Strength): {cramers_v}")

Cramér's V (Strength): 0.030982086888879927


In [186]:
# Trying to group categories of payer_code column

In [187]:
# Create a mapping dictionary
payer_map = {
    'MC': 'Public', 'MD': 'Public',
    'SP': 'Self-Pay', 'UN': 'Self-Pay',
    'BC': 'Private', 'HM': 'Private', 'CP': 'Private', 'CM': 'Private',
    'OG': 'Private', 'PO': 'Private', 'DM': 'Private', 'CH': 'Private',
    'WC': 'Private', 'OT': 'Private', 'MP': 'Private', 'SI': 'Private',
    'Unknown': 'Unknown', 'FR': 'Unknown'
}

df['payer_grouped'] = df['payer_code'].map(payer_map)

In [188]:
# running 'chi2' and 'Cramér's V' on grouped column

In [189]:
# Create a cross-tabulation table
contingency_table = pd.crosstab(df['payer_grouped'], df['readmitted_binary'])

# Run the test
chi2, p, dof, ex = chi2_contingency(contingency_table)
print(f"P-value: {p}")

n = df.shape[0]  # total observations
min_dim = min(contingency_table.shape) - 1
cramers_v = np.sqrt(chi2 / (n * min_dim))

print(f"Cramér's V (Strength): {cramers_v}")

P-value: 6.775612174625376e-14
Cramér's V (Strength): 0.02515431148217195


In [190]:
# No further improvement , and decided to drop the column

df.drop(columns=['payer_code','payer_grouped'], inplace=True)

### Cleaning A1Cresult

In [191]:
df['A1Cresult'].value_counts()

A1Cresult
NotTested    84745
>8            8216
Norm          4990
>7            3812
Name: count, dtype: int64

In [192]:
# converting it to ordinal

# NotTested → 0
# Norm → 1
# >7 → 2
# >8 → 3

In [193]:
A1Cresult_map = {
  "NotTested" : 0,
  "Norm" : 1,
  ">7" : 2,
  ">8" : 3
}

df['A1Cresult'] = df['A1Cresult'].map(A1Cresult_map)

In [194]:
df['A1Cresult'].value_counts()

A1Cresult
0    84745
3     8216
1     4990
2     3812
Name: count, dtype: int64

### Checking Categorical Columns

In [214]:
df['diag_1'].nunique()

11

In [215]:
df['diag_2'].nunique()

11

In [216]:
df['diag_3'].nunique()

11

In [ ]:
df['admission_type'].nunique()

5

In [ ]:
df['discharge_disposition'].nunique()

16

In [ ]:
df['admission_source'].nunique()

9

In [ ]:
df['medical_specialty'].nunique()

16

In [ ]:
# None of tham need to be grouped , it can easly encoded using OneHotEncoding now

### Saving clean file before apply encoding

In [218]:
df.to_csv("./data/clean/day-28.csv", index=False)

### Making Preprocessor 

In [228]:
from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import joblib

In [222]:
# Train-Test Split

X = df.drop(columns=['readmitted_binary'])
y = df['readmitted_binary']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [225]:
# Build Preprocessor

categorical_cols = [
    'race', 'gender', 'admission_type',
    'discharge_disposition', 'admission_source',
    'medical_specialty', 'diag_1', 'diag_2', 'diag_3'
]

numeric_cols = X_train.select_dtypes(include=['int64']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
    ]
)

In [226]:
# Fit on training data

preprocessor.fit(X_train)

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``. e.g. `

In [230]:
# Save Preprocessor

joblib.dump(preprocessor, "./data/clean/preprocessing_pipeline.pkl")

['./data/clean/preprocessing_pipeline.pkl']